In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

In [23]:
cust_data = pd.read_csv('data_transform.csv')
cust_data.shape

(100233, 206)

In [24]:
cust_data.head()

,ACTL_FMLY_NUM,AGE,ARPU,AUTR_FAIL_MCNT,AUTR_FAIL_YN,AVG_CALL_FREQ,AVG_CALL_TIME,AVG_STLN_RATE,CB_GUIF_AMT,CB_GUIF_CNT,...,MATE_OCCP_NAME_C_07,MATE_OCCP_NAME_C_08,MATE_OCCP_NAME_C_09,MATE_OCCP_NAME_C_10,MATE_OCCP_NAME_C_11,MATE_OCCP_NAME_C_12,MATE_OCCP_NAME_C_13,MATE_OCCP_NAME_C_14,MATE_OCCP_NAME_C_15,MATE_OCCP_NAME_C_16
0,4,50,30000,10,1.0,493,450,0,420001,3,...,0,0,0,0,0,0,0,0,1,0
1,4,50,30000,0,0.0,22,81,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,60,30000,0,0.0,17,139,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,35,30000,0,0.0,0,1118,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,45,50000,0,0.0,354,396,95,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

## 연체자 data duplicate

In [26]:
cust = cust_data[cust_data['TARGET'] == 0]
cust_overdue = cust_data[cust_data['TARGET'] == 1]     # 연체자

print(cust.shape[0])
print(cust_overdue.shape[0])

95946
4287


In [27]:
variables = cust_data.drop('TARGET', 1).columns
variables

Index(['ACTL_FMLY_NUM', 'AGE', 'ARPU', 'AUTR_FAIL_MCNT', 'AUTR_FAIL_YN',
       'AVG_CALL_FREQ', 'AVG_CALL_TIME', 'AVG_STLN_RATE', 'CB_GUIF_AMT',
       'CB_GUIF_CNT',
       ...
       'MATE_OCCP_NAME_C_07', 'MATE_OCCP_NAME_C_08', 'MATE_OCCP_NAME_C_09',
       'MATE_OCCP_NAME_C_10', 'MATE_OCCP_NAME_C_11', 'MATE_OCCP_NAME_C_12',
       'MATE_OCCP_NAME_C_13', 'MATE_OCCP_NAME_C_14', 'MATE_OCCP_NAME_C_15',
       'MATE_OCCP_NAME_C_16'],
      dtype='object', length=205)

### ML modeling

In [28]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1)
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [29]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 205)
(3439,)


In [30]:
# clf = GradientBoostingClassifier(n_estimators=500, random_state=0)

In [31]:
max_epoch = 10

for i in range(max_epoch+1):
#     x2_train, x2_test, y2_train, y2_test = train_test_split(x_train, y_train, test_size=0.2)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    
    print('========================================= ', i)
    print('test {} : test overdue {}'.format(y2_test.shape[0], sum(y2_test == 1)))
    print('')
    
    clf = GradientBoostingClassifier(n_estimators=500, random_state=0)
    clf.fit(x_train, y_train)
    
#     y_pred = clf.predict(x2_test)
#     model_performance(y2_test, y_pred)
    
    varDic = {'var':variables, 'importance':clf.feature_importances_}
    importance = pd.DataFrame(varDic)
    print(importance.sort_values(by='importance', ascending=False).head(10))

    x_train = pd.concat([x_train, x_overdue])
    y_train = pd.concat([y_train, y_overdue])

    print('------------------------')
    y_pred_0 = clf.predict(x_test)
    model_performance(y_test, y_pred_0)
    print('------------------------')


=========================================  0
test 16038 : test overdue 688

     importance              var
88     0.064612         dv_22_23
135    0.044719         dv_65_66
141    0.035103         dv_70_71
38     0.027526         LNIF_CNT
71     0.022667     TOT_LOAN_CNT
125    0.021641         dv_56_57
61     0.020762  SPTCT_OCCR_MDIF
16     0.020035    CRDT_CARD_CNT
87     0.019644         dv_21_22
142    0.017718         dv_71_72
------------------------
confusion matrix
[[19086   132]
 [  619   210]]
accuracy : 0.963
precision : 0.614
recall : 0.253
F1 : 0.359
------------------------
=========================================  1
test 16038 : test overdue 688

     importance              var
88     0.058073         dv_22_23
135    0.048232         dv_65_66
141    0.030994         dv_70_71
38     0.028916         LNIF_CNT
125    0.024989         dv_56_57
71     0.020052     TOT_LOAN_CNT
61     0.018929  SPTCT_OCCR_MDIF
130    0.018128         dv_60_61
43     0.017401    LT1Y_MXOD_

KeyboardInterrupt: 